In [3]:
import os
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import r2_score
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.auto import tqdm, trange

tqdm.pandas()

from collections import defaultdict

local_root_path = "."

sys.path.append(local_root_path)
import annutils
from collections import defaultdict

In [12]:
num_sheets = 9

observed_stations_ordered_by_median = ['RSMKL008', 'RSAN032', 'RSAN037', 'RSAC092', 'SLTRM004', 'ROLD024',
                                       'CHVCT000', 'RSAN018', 'CHSWP003', 'CHDMC006', 'SLDUT007', 'RSAN072',
                                       'OLD_MID', 'RSAN058', 'ROLD059', 'RSAN007', 'RSAC081', 'SLMZU025',
                                       'RSAC075', 'SLMZU011', 'SLSUS012', 'SLCBN002', 'RSAC064']

output_stations = ['CHDMC006-CVP INTAKE', 'CHSWP003-CCFB_INTAKE', 'CHVCT000-VICTORIA INTAKE',
                   'OLD_MID-OLD RIVER NEAR MIDDLE RIVER', 'ROLD024-OLD RIVER AT BACON ISLAND',
                   'ROLD059-OLD RIVER AT TRACY BLVD', 'RSAC064-SACRAMENTO R AT PORT CHICAGO',
                   'RSAC075-MALLARDISLAND', 'RSAC081-COLLINSVILLE', 'RSAC092-EMMATON',
                   'RSAC101-SACRAMENTO R AT RIO VISTA', 'RSAN007-ANTIOCH', 'RSAN018-JERSEYPOINT',
                   'RSAN032-SACRAMENTO R AT SAN ANDREAS LANDING', 'RSAN037-SAN JOAQUIN R AT PRISONERS POINT',
                   'RSAN058-ROUGH AND READY ISLAND', 'RSAN072-SAN JOAQUIN R AT BRANDT BRIDGE',
                   'RSMKL008-S FORK MOKELUMNE AT TERMINOUS', 'SLCBN002-CHADBOURNE SLOUGH NR SUNRISE DUCK CLUB',
                   'SLDUT007-DUTCH SLOUGH', 'SLMZU011-MONTEZUMA SL AT BELDONS LANDING',
                   'SLMZU025-MONTEZUMA SL AT NATIONAL STEEL', 'SLSUS012-SUISUN SL NEAR VOLANTI SL',
                   'SLTRM004-THREE MILE SLOUGH NR SAN JOAQUIN R', 'SSS-STEAMBOAT SL', 'CCW-MIDDLE RIVER INTAKE',
                   'OH4-OLD R @ HWY 4', 'SLRCK005-CCWD_Rock', 'MRU-MIDDLE RIVER AT UNDINE ROAD', 'HLL-HOLLAND TRACT',
                   'BET-PIPER SLOUGH @ BETHEL TRACT', 'GES-SACRAMENTO R BELOW GEORGIANA SLOUGH',
                   'NMR: N FORK MOKELUMNE R NEAR WALNUT GROVE', 'IBS-CORDELIA SLOUGH @ IBIS CLUB',
                   'GYS-GOODYEAR SLOUGH AT MORROW ISLAND CLUB', 'BKS-SLBAR002-North Bay Aqueduct/Barker Sl']

output_stations, name_mapping = annutils.read_output_stations(output_stations, observed_stations_ordered_by_median)

key_stations = ['RSAC064','CCWD_Rock','CHDMC006', 'CHSWP003','RSAC092','RSAN018']
key_station_mappings = {'CCWD Rock': 'RSL',
                        'Middle River Intake':'MUP',
                        'Old River Hwy 4': 'OH4',
                        'OLD MID':'OLD_MID'}

In [4]:
def evaluate_sequences(target, pred, metrics,print_details=False):
    assert len(target) == len(pred), 'Target and predicted sequence length must equal.'
    valid_entries = target>0
    sequence_length = np.sum(valid_entries)
    # print('Total samples: %d, valid samples: %d' % (len(target), np.sum(valid_entries)))
    if np.any(sequence_length == 0):
        return {k: 0 for k in metrics}
    target=target[valid_entries]
    pred = pred[valid_entries]
    SD_pred = np.sqrt( np.sum((pred-np.mean(pred)) ** 2) /(sequence_length-1))
    SD_target = np.sqrt( np.sum((target-np.mean(target)) ** 2) /(sequence_length-1))

    eval_results = defaultdict(float)

    for m in metrics:
        if m =='MSE':
            eval_results[m] = ((target - pred)**2).mean()
        elif m =='Bias':
            eval_results[m] = np.sum(pred - target)/np.sum(target) * 100
        elif m == 'R':
            eval_results[m] = np.sum(np.abs((pred-np.mean(pred)) * (target - np.mean(target)))) / (sequence_length * SD_pred * SD_target)
        elif m == 'RMSD':
            eval_results[m] = np.sqrt(np.sum( ( ( pred-np.mean(pred) ) * ( target - np.mean(target) ) ) ** 2 ) / sequence_length)
        elif m == 'NSE':
            eval_results[m] = 1 - np.sum( ( target - pred ) ** 2 ) / np.sum( (target - np.mean(target) ) ** 2 )
    if print_details:
        print('(sum(pred - mean(pred)) x (target - mean(target))) =  %.4f' % np.sum((pred-np.mean(pred)) * (target - np.mean(target))))
        print('MSE =  %.4f' % eval_results[m])
        print('sum(pred - target) = %.4f' % np.sum(pred - target))
        print('sum(target) = %.4f' % np.sum(target))
        print('target standard deviation = %.6f, prediction standard deviation =%.6f' %(SD_target,SD_pred))
    return eval_results

In [17]:
compression_opts = dict(method='zip', archive_name='out.csv')

experiment = "6years"

experiment_dir = os.path.join(local_root_path, "Experiments", experiment)
if not os.path.exists(experiment_dir):
    print("Experiment does not exist")


model_dir = os.path.join(experiment_dir, "models")

model_file = "mtl_i118_residual_lstm_8_2.h5"

model_name = os.path.splitext(model_file)[0]

model_prediction_dir = os.path.join(experiment_dir, "results", "prediction", model_name)
if not os.path.exists(model_prediction_dir):
    print("model_prediction_dir does not exist")

file_name = "dsm2_ann_inputs_base.csv"
prediction_file = os.path.join(model_prediction_dir, file_name)
if not os.path.exists(prediction_file):
    print("prediction_file does not exist")

file_part = os.path.splitext(file_name)[0]

target_file = os.path.join(experiment_dir, "results", "target", file_part + "_target.csv")
if not os.path.exists(target_file):
    print("target_file does not exist")

metrics = ['MSE', 'Bias', 'R', 'RMSD', 'NSE']

full_results={}
prediction = pd.read_csv(prediction_file, compression=compression_opts, index_col=0)
target = pd.read_csv(target_file, compression=compression_opts, index_col=0)

# for ii, location in enumerate(output_stations):
ii = 0
location = "CHDMC006"

print("Evaluating %s %s" % (ii, location))

if any([k.lower() in location.lower() for k in key_stations]):
    print_details = True
else:
    print_details = False


# compute training results
target_for_station = target.iloc[:, ii]
prediction_for_station = prediction.iloc[:, ii]
train_results = evaluate_sequences(target_for_station, prediction_for_station, metrics, print_details=print_details)
train_results['R^2'] = r2_score(prediction_for_station, target_for_station)
full_results['%s_train' %location] = train_results


Evaluating 0 CHDMC006
(sum(pred - mean(pred)) x (target - mean(target))) =  274026792.3418
MSE =  0.7288
sum(pred - target) = -157030.2967
sum(target) = 4642344.7350
target standard deviation = 169.762621, prediction standard deviation =171.816666
